In [ ]:
#---Installing Libraries and Dependencies---
import subprocess
import sys

def install_dependencies():
    # List of pip packages
    packages = [
        "openai",
        "networkx",
        "ipycytoscape>=1.3.1",
        "ipywidgets",
        "pandas",
        "together",
        "rdflib",
        "google-search-results",
        "PyPDF2",
        "pytesseract",
        "pdf2image",
        "Pillow"
    ]

    print("Installing pip packages...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", *packages])

    print("Installing apt packages (poppler-utils)...")
    subprocess.check_call(["apt-get", "update"])
    subprocess.check_call(["apt-get", "install", "-y", "poppler-utils"])

install_dependencies()


Installing pip packages...
Installing apt packages (poppler-utils)...


In [ ]:
#---Retrieving Credentials---
def retrieving_credentials():

    #---Storing API Credentials---
    #os.environ["OPENAI_API_KEY"] = 'yourkey'
    #os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"

    os.environ["TOGETHER_API_KEY"] = 'yourkey'
    #os.environ["TOGETHER_API_BASE"] = "https://api.together.xyz/v1"



    #os.environ["DEEEPSEEK_API_KEY"] = 'yourkey'
    #os.environ["DEEPSEEK_API_BASE"] = "https://api.deepseek.com"'''

    #---LLM model selection---

    llm_model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

    print(f"Intended LLM model: {llm_model_name}")

    #---Retrieving  Credentials---

    api_key = os.getenv("TOGETHER_API_KEY")
    base_url = os.getenv("TOGETHER_API_BASE")

    print(f"Retrieved API key: {'Set' if api_key else 'Not_set'}")
    print(f"Retrieved Base URL: {base_url if base_url else 'Not Set (will use default TogetherAI)'}")

    return api_key, base_url, llm_model_name


In [ ]:
#---Client response check---
def client_response_check():
    from together import Together

    client = Together()

    response = client.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=[{"role": "user", "content": "Type LLM model is online only!"}]
    )
    print(response.choices[0].message.content)

    return client, response

In [ ]:
#---Initializing The Client---
def initialize_client(api_key, base_url, client):
    # --- Validate Key and Initialize Client ---
    from together import Together

    if not api_key:
        print("Error: TOGETHER_API_KEY environment variable not set or key not provided directly.")
        print("Please set the environment variable (or uncomment/edit the test lines) and restart the kernel.")
        raise SystemExit("API Key configuration failed.")
    else:
        try:
            client = Together(api_key=api_key, base_url=base_url)
            print("Together client initialized successfully.")
        except Exception as e:
            print(f"Error initializing Together client: {e}")
            print("Check your API key, base URL (if used), and network connection.")
            raise SystemExit("LLM client initialization failed.")

    return client


In [ ]:
#---Defining the LLM Parameters---
def llm_parameters():
    #---Define LLM Call Parameters---
    llm_temperature = 0.0 # Lower temperature for more deterministic, factual output. 0.0 is best for extraction.
    llm_max_tokens =  4096 # Max tokens for the LLM response (adjust based on the model limits)

    print(f"LLM Temperature set to: {llm_temperature}")
    print(f"LLM Max Tokens set to: {llm_max_tokens}")

    return llm_temperature, llm_max_tokens


In [ ]:
#---Searching for the relevant documents---
def search_satellite_power_docs(query, num_results=10):
    params = {
        "engine": "google",
        "q": query,
        "num": num_results,
        "api_key": "yourserpapikey"
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    links = [res["link"] for res in results.get("organic_results", [])]
    return links





In [ ]:
#---Ranking the topic links----
def rank_topic_links(links, base_topic = "satellite power system fault diagnosis"):
    ranking_model = SentenceTransformer('all-MiniLM-L6-v2')
    base_embed = ranking_model.encode(base_topic, convert_to_tensor=True)
    scored_links = []

    for link in links:
        try:
            response = requests.get(link, timeout=10)
            soup = BeautifulSoup(response.text, "html.parser")
            paragraphs = soup.get_text()

            # Embed the full text of the page
            doc_embed = ranking_model.encode(paragraphs, convert_to_tensor=True)

            # Calculate cosine similarity
            score = util.pytorch_cos_sim(base_embed, doc_embed).item()

            # Optional: use your Together client for extra LLM verification if needed
            # response = client.chat.completions.create(
            #     model="mistralai/Mixtral-8x7B-Instruct-v0.1",
            #     messages=[
            #         {"role": "user", "content": f"Does this document relate to {base_topic}? Content:\n{paragraphs[:1000]}"}
            #     ]
            # )
            # confidence_verbal = response.choices[0].message.content

            scored_links.append((link, score))

        except Exception as e:
            print(f"Failed to process {link}: {e}")
            continue

    # Sort links by descending score
    return sorted(scored_links, key=lambda x: x[1], reverse=True)

In [ ]:
#---Text Extraction---
def extract_text_from_pdf(pdf_url):
    # Download the PDF
    response = requests.get(pdf_url)
    with open("temp.pdf", "wb") as f:
        f.write(response.content)

    # First try to extract text normally
    reader = PyPDF2.PdfReader("temp.pdf")
    all_text = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            all_text += text + "\n"

    # If no text found, fallback to OCR on images of each page
    if not all_text.strip():
        print("No text found in PDF pages, running OCR...")
        images = convert_from_path("temp.pdf")
        ocr_text = ""
        for img in images:
            ocr_text += pytesseract.image_to_string(img) + "\n"
        return ocr_text

    return all_text


def extract_text_from_html(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.get_text()


In [ ]:
# -- Refactored unstructured_text() --
def unstructured_text(para):
    print("--- Input Text Loaded ---")
    print(para)
    print("-" * 25)

    # Basic Statistics
    char_count = len(para)
    word_count = len(para.split())
    print(f"Total characters: {char_count}")
    print(f"Approximate word count: {word_count}")
    print("-" * 25)

    return para, char_count, word_count


In [ ]:
#--- Document Collection Pipeline---
def document_collection_pipeline_main():
    query = "satellite power system fault diagnosis PDF"
    links = search_satellite_power_docs(query)
    ranked_links = rank_topic_links(links, base_topic="satellite power system fault diagnosis")

    collected_texts = []
    num_top = max(1, int(len(ranked_links) * 0.8))

    for link, score in ranked_links[:num_top]:
        print(f"\nFetching: {link} | Confidence Score: {score}")
        try:
            if link.endswith(".pdf"):
                doc_text = extract_text_from_pdf(link)
            else:
                doc_text = extract_text_from_html(link)

            for para in doc_text.split("\n\n"):
                if len(para.strip()) > 50:
                    text, char_count, word_count = unstructured_text(para)
                    collected_texts.append(text)

        except Exception as e:
            print(f"Error processing link: {link}\n{e}")
            continue

    full_text = "\n".join(collected_texts)

    # Token-safe chunking (e.g., split by ~2000 words)
    CHUNK_WORDS = 1500
    words = full_text.split()
    full_text_chunks = [" ".join(words[i:i+CHUNK_WORDS]) for i in range(0, len(words), CHUNK_WORDS)]

    return full_text_chunks


In [ ]:
#--- Co-refrence resolution prompting---
def coref_prompt(text_chunk, client, llm_model_name, llm_temperature, llm_max_tokens):
    prompt = f"""
    Resolve all coreferences in the following paragraph. Replace all pronouns (like "she", "he", "they") or whatever pronoun is in use, with the actual names they refer to. Do not summarize:

    Text:
    \"\"\"{text_chunk}\"\"\"

    Output:
    """

    response = client.chat.completions.create(
        model=llm_model_name,
        messages=[
            {"role": "system", "content": "You are a helpful AI trained in language understanding."},
            {"role": "user", "content": prompt}
        ],
        temperature=llm_temperature,
        max_tokens=llm_max_tokens
    )

    resolved_text = response.choices[0].message.content.strip()
    return resolved_text


In [ ]:
#--- Chunk Configuration---
def chunk_configuration(resolved_text):

    #--- Chunking Configuration ---
    chunk_size = 150 #Number of words per chunk (adjust as needed)
    overlap = 30 # Number of words to overlap (must be < chunk_size)

    print(f"Chunk Size set to: {chunk_size} words")
    print(f"Overlap set to: {overlap} words")

    #--- Basic Validation ---
    if overlap >= chunk_size and chunk_size > 0:
      print(f"Error: Overlap ({overlap}) must be smaller than chunk size ({chunk_size}).")
      raise SystemExit("Chunking configuration error.")
    else:
      print("Chunking configuration is valid.")
    words = resolved_text.split()
    total_words = len(words)

    print(f"Text split into {total_words} words.")

    #---Visualization of first 20 words---
    print(f"First 20 words: {words[:20]}")

    return chunk_size, overlap, words, total_words


In [ ]:
#---Chunking Execution---
def chunking_execution(chunk_size, overlap, words, total_words):
    #---performing chunking based on the configuration---

    chunks = []
    start_index = 0
    chunk_number = 1

    print(f"Starting chunking process...")
    while start_index < total_words:
      end_index = min(start_index + chunk_size, total_words)
      chunk_text = " ".join(words[start_index:end_index])
      chunks.append({"text": chunk_text, "chunk_number": chunk_number})

      #Printing the detailed log

      print(f" Created chunk {chunk_number}: words {start_index} to {end_index}")
      next_start_index = start_index + chunk_size - overlap

      # Ensure progress is made
      if next_start_index <= start_index:
            if end_index == total_words:
                break #Already processed the last part
            next_start_index = start_index + 1

      start_index = next_start_index
      chunk_number += 1

      #---Safety Break---
      if chunk_number > total_words: #Simple safety
            print("Warning: Chunking loop exceeded total word count, breaking.")
            break

    print(f"\nText successfully split into {len(chunks)} chunks.")

    print("--- Chunk Details ---")
    if chunks:
      # Create a DataFrame for better visualization
      chunks_df = pd.DataFrame(chunks)
      chunks_df['word_count'] = chunks_df['text'].apply(lambda x: len(x.split()))
      display(chunks_df[['chunk_number', 'word_count', 'text']])
    else:
      print("No chunks were created (text might be shorter than chunk size).")
    print("-"*25)

    return chunks, chunks_df

In [ ]:
#---Extraction Prompting---
def extraction_prompt(chunks):

    # --- System Prompt: Sets the context/role for the LLM ---
    extraction_system_prompt = """
    You are an AI expert specialized in knowledge graph extraction.
    Your task is to identify and extract factual Subject-Predicate-Object (SPO) triples from the given text.
    When temporal information (e.g., years, dates, or time periods) is explicitly or implicitly available, include it as an additional "temporal" key in each triple.
    Focus on accuracy and adhere strictly to the JSON output format requested in the user prompt.
    Extract core entities and the most direct relationship.
    """


    extraction_user_prompt_template = """
    Please extract Subject-Predicate-Object (S-P-O) triples from the text below.

    **VERY IMPORTANT RULES:**
    1.  **Output Format:** Respond ONLY with a single, valid JSON array. Each element MUST be an object with keys "subject", "predicate", "object". If a time or date is mentioned in relation to the triple, add a fourth key "temporal".
    2.  **JSON Only:** Do NOT include any text before or after the JSON array. Do NOT use markdown ```json ... ``` tags.
    3.  **Concise Predicates:** Keep the 'predicate' value concise (1-3 words, ideally 1-2). Use verbs or short verb phrases (e.g., 'discovered', 'was born in', 'won').
    4.  **Lowercase:** ALL values for 'subject', 'predicate', 'object', and 'temporal' MUST be lowercase.
    5.  **Pronoun Resolution:** Replace pronouns (she, he, it, her, etc.) with the specific lowercase entity name they refer to based on the text context (e.g., 'marie curie').
    6.  **Specificity:** Capture specific details (e.g., 'nobel prize in physics' instead of just 'nobel prize' if specified).
    7.  **Completeness:** Extract all distinct factual relationships mentioned.
    8.  **Temporal Info:** If a specific time, date, or year is mentioned (e.g., "in 1903", "on july 4", "during the 20th century"), include it as a "temporal" key in the JSON object.

    **Text to Process:**
    {text_chunk}

    Required JSON Output Format Example:
    [
      {{ "subject": "marie curie", "predicate": "discovered", "object": "radium", "temporal": "1898" }},
      {{ "subject": "marie curie", "predicate": "won", "object": "nobel prize in physics", "temporal": "1903" }}
    ]

    Your JSON Output (MUST start with '[' and end with ']'):
    You MUST return an array of JSON objects (even if there's only one). Wrap it in [ ] brackets.
    Only include the "temporal" key if time info is actually present or implied.
    """

    print("--- System Prompt ---")
    print(extraction_system_prompt)
    print("\n" + "-" * 25 + "\n")

    print("--- User Prompt Template (Structure) ---")
    # Show structure, replacing the placeholder for clarity
    print(extraction_user_prompt_template.replace("{text_chunk}", "[... text chunk goes here ...]"))
    print("\n" + "-" * 25 + "\n")

    # Show an example of the *actual* prompt that will be sent for the first chunk
    print("--- Example Filled User Prompt (for Chunk 1) ---")
    if chunks:
        example_filled_prompt = extraction_user_prompt_template.format(text_chunk=chunks[0]['text'])
        # Displaying a limited portion for brevity
        print(example_filled_prompt[:600] + "\n[... rest of chunk text ...]\n" + example_filled_prompt[-200:])
    else:
        print("No chunks available to create an example filled prompt.")
    print("\n" + "-" * 25)

    return extraction_system_prompt, extraction_user_prompt_template



In [ ]:
#---S-P-O Extraction---
def spo_extraction(extraction_user_prompt_template, extraction_system_prompt, chunks, llm_model_name, llm_temperature, llm_max_tokens, client):
    import json
    import re
    import pandas as pd

    all_extracted_triples = []
    failed_chunks = []

    print(f"Starting triple extraction from {len(chunks)} chunks using model '{llm_model_name}'...")

    for chunk_info in chunks:  # <-- Loop through all chunks
        chunk_text = chunk_info['text']
        chunk_num = chunk_info['chunk_number']

        print(f"\n--- Processing Chunk {chunk_num}/{len(chunks)} ---")

        print("1. Formatting User Prompt...")
        user_prompt = extraction_user_prompt_template.format(text_chunk=chunk_text)

        llm_output = None
        error_message = None

        try:
            print("2. Sending request to LLM...")
            response = client.chat.completions.create(
                model=llm_model_name,
                messages=[
                    {"role": "system", "content": extraction_system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=llm_temperature,
                max_tokens=llm_max_tokens,
                response_format={ "type": "json_object" },
            )
            print("   LLM response received.")

            print("3. Extracting raw response content...")
            llm_output = response.choices[0].message.content.strip()
            print(f"--- Raw LLM Output (Chunk {chunk_num}) ---")
            print(llm_output)
            print("-" * 20)

        except Exception as e:
            error_message = str(e)
            print(f"   ERROR during API call: {error_message}")
            failed_chunks.append({'chunk_number': chunk_num, 'error': f'API/Processing Error: {error_message}', 'response': ''})
            continue  # Skip parsing and validation for this chunk

        # 4. Parse JSON
        parsed_json = None
        parsing_error = None
        if llm_output is not None:
            print("4. Attempting to parse JSON from response...")
            try:
                parsed_data = json.loads(llm_output)
                if isinstance(parsed_data, dict):
                    print("   Detected dictionary response, attempting to extract list...")
                    list_values = [v for v in parsed_data.values() if isinstance(v, list)]
                    if len(list_values) == 1:
                        parsed_json = list_values[0]
                        print("      Successfully extracted list from dictionary.")
                    else:
                        raise ValueError("JSON object received, but doesn't contain a single list of triples.")
                elif isinstance(parsed_data, list):
                    parsed_json = parsed_data
                    print("   Successfully parsed JSON list directly.")
                else:
                    raise ValueError("Parsed JSON is not a list or expected dictionary wrapper.")
            except json.JSONDecodeError as json_err:
                parsing_error = f"JSONDecodeError: {json_err}. Trying regex fallback..."
                print(f"   {parsing_error}")
                match = re.search(r'^\s*(\[.*?\])\s*$', llm_output, re.DOTALL)
                if match:
                    try:
                        parsed_json = json.loads(match.group(1))
                        print("      Successfully parsed JSON from regex extraction.")
                        parsing_error = None
                    except json.JSONDecodeError as nested_err:
                        parsing_error = f"JSONDecodeError after regex: {nested_err}"
                        print(f"      ERROR: Regex content is not valid JSON: {nested_err}")
                else:
                    parsing_error = "JSONDecodeError and Regex fallback failed."
                    print("      ERROR: Regex could not find JSON array structure.")
            except ValueError as val_err:
                parsing_error = f"ValueError: {val_err}"
                print(f"   ERROR: {parsing_error}")

        if parsed_json is not None:
            print(f"--- Parsed JSON Data (Chunk {chunk_num}) ---")
            print(json.dumps(parsed_json, indent=2))
            print("-" * 20)
        else:
            print(f"--- JSON Parsing FAILED (Chunk {chunk_num}) --- ")
            print(f"   Final Parsing Error: {parsing_error}")
            print("-" * 20)
            failed_chunks.append({'chunk_number': chunk_num, 'error': f'Parsing Failed: {parsing_error}', 'response': llm_output})
            continue

        # 5. Validate and store
        print("5. Validating structure and extracting triples...")
        valid_triples_in_chunk = []
        invalid_entries = []

        if isinstance(parsed_json, list):
            for item in parsed_json:
                if isinstance(item, dict) and all(k in item for k in ['subject', 'predicate', 'object']):
                    if all(isinstance(item[k], str) for k in ['subject', 'predicate', 'object']):
                        item['chunk'] = chunk_num
                        valid_triples_in_chunk.append(item)
                    else:
                        invalid_entries.append({'item': item, 'reason': 'Non-string value'})
                else:
                    invalid_entries.append({'item': item, 'reason': 'Incorrect structure/keys'})
        else:
            invalid_entries.append({'item': parsed_json, 'reason': 'Not a list'})
            if not any(fc['chunk_number'] == chunk_num for fc in failed_chunks):
                failed_chunks.append({'chunk_number': chunk_num, 'error': 'Parsed data not a list', 'response': llm_output})

        print(f"   Found {len(valid_triples_in_chunk)} valid triples in this chunk.")
        if invalid_entries:
            print(f"   Skipped {len(invalid_entries)} invalid entries.")

        if valid_triples_in_chunk:
            print(f"--- Valid Triples Extracted (Chunk {chunk_num}) ---")
            display(pd.DataFrame(valid_triples_in_chunk))
            print("-" * 20)
            all_extracted_triples.extend(valid_triples_in_chunk)
        else:
            print(f"--- No valid triples extracted from this chunk. ---")
            print("-" * 20)

        print(f"--- Running Total Triples Extracted: {len(all_extracted_triples)} ---")
        print(f"--- Failed Chunks So Far: {len(failed_chunks)} ---")

    print("\n Finished processing all chunks.")
    if failed_chunks:
        print("\n--- Failed Chunks Summary ---")
        for failed in failed_chunks:
            print(f"Chunk #{failed['chunk_number']} failed with error: {failed['error']}")
            print(f"Response:\n{failed['response']}\n")
    else:
        print("\n No failed chunks!")

    return all_extracted_triples, failed_chunks


In [ ]:
#---Extraction and simplification of SPO Triples---
def extract_and_simplify_triples(all_extracted_triples, client, llm_model_name, llm_temperature, llm_max_tokens):

    #Creating the variable
    all_final_triples = []


    # STEP 1: Extract complex triples
    complex_system_prompt = """You are an assistant that simplifies complex SPO (Subject–Predicate–Object) triples into smaller, atomic relationships without losing their intended meaning or logical structure.

A triple is considered **complex** and needs simplification if:

1. The **predicate** contains multiple actions, modifiers, or logical connectors (e.g., "owns and operates", "is responsible for and manages").
2. The **subject** or **object** is a compound concept (e.g., "economic and political conditions", "intrinsic metabolic and structural state").
3. The triple **implicitly encodes multiple relationships**, such as ownership, role, category, or part-whole associations.
4. Any **part of the triple** is vague or underspecified (e.g., "None", "this", "that process").
5. There is a **temporal aspect** (e.g., dates, durations, time ranges) that applies to the subject, predicate, or object.

Your task is to:
- Decompose compound predicates into distinct triples, each representing a single relationship.
- Split compound subjects or objects into multiple entities if each part plays a different role.
- Make implicit relationships explicit, such as "is a part of", "is a type of", or "belongs to".
- Replace vague entities (e.g., "None", "this") with inferred concrete references where context allows.
- Identify any **temporal expressions** (e.g., "in 1995", "since 2020", "during the war") and extract them as separate triples using predicates such as:
  - "happened in"
  - "started in"
  - "ended in"
  - "occurred during"
  - "was active during"

###  Examples of complex triples and how to simplify them:

**Example 1 (compound predicate):**
- Complex: { "subject": "john", "predicate": "owns and operates", "object": "two factories" }
- Simplified:
  - { "subject": "john", "predicate": "owns", "object": "two factories" }
  - { "subject": "john", "predicate": "operates", "object": "two factories" }

**Example 2 (compound subject):**
- Complex: { "subject": "economic and political conditions", "predicate": "influence", "object": "public policy" }
- Simplified:
  - { "subject": "economic conditions", "predicate": "influence", "object": "public policy" }
  - { "subject": "political conditions", "predicate": "influence", "object": "public policy" }

**Example 3 (implicit parts):**
- Complex: { "subject": "biological cell", "predicate": "has a membrane and nucleus", "object": null }
- Simplified:
  - { "subject": "biological cell", "predicate": "has", "object": "membrane" }
  - { "subject": "biological cell", "predicate": "has", "object": "nucleus" }

**Example 4 (role inference):**
- Complex: { "subject": "employee", "predicate": "is the manager of", "object": "team" }
- Simplified:
  - { "subject": "employee", "predicate": "has role", "object": "manager" }
  - { "subject": "manager", "predicate": "manages", "object": "team" }

**Example 5 (vague subject):**
- Complex: { "subject": "none", "predicate": "damages", "object": "machine" }
- Simplified:
  - Try to resolve or infer the missing subject from context if possible.
  - If not resolvable, flag for human review.

**Example 6 (temporal factor):**
- Complex: { "subject": "marie curie", "predicate": "won", "object": "nobel prize in physics in 1903" }
- Simplified:
  - { "subject": "marie curie", "predicate": "won", "object": "nobel prize in physics" }
  - { "subject": "marie curie winning nobel prize in physics", "predicate": "happened in", "object": "1903" }

Make sure every triple is atomic, clear, and temporally enriched if time context is available.
"""


    complex_user_prompt = f"""Here is a list of triples extracted from a document.

Your job is to **select only the complex triples** from this list, using the rules described (including compound, implicit, vague, and temporal relationships):

{json.dumps(all_extracted_triples, indent=2)}

Return only the ones that need simplification in JSON.

These are the complex triples:
[
  {{ "subject": "john", "predicate": "owns and operates", "object": "two factories" }},
  ...
]
"""




    # STEP 1.5: Extract JSON from response safely using regex
    # Remove leading "These are the complex triples:" or similar headers

    def extract_json_list(text):
        try:
            return json.loads(text)
        except:
            fenced_match = re.search(r"```(?:json)?\s*(\[\s*{.*?}\s*\])\s*```", text, re.DOTALL)
            if fenced_match:
                try:
                    return json.loads(fenced_match.group(1))
                except Exception as e:
                    print(" Failed parsing fenced JSON block:", e)
                    print("Matched text:\n", fenced_match.group(1))

            loose_match = re.search(r"(\[\s*{.*?}\s*\])", text, re.DOTALL)
            if loose_match:
                try:
                    return json.loads(loose_match.group(1))
                except Exception as e:
                    print(" Failed parsing loose JSON match:", e)
                    print("Matched text:\n", loose_match.group(1))

            raise ValueError("No valid JSON list found.")

    response = client.chat.completions.create(
        model=llm_model_name,
        messages=[
            {"role": "system", "content": complex_system_prompt},
            {"role": "user", "content": complex_user_prompt}
        ],
        temperature=llm_temperature,
        max_tokens=llm_max_tokens
    )


    try:
        complex_triples = extract_json_list(response.choices[0].message.content)
    except Exception as e:
        print("\n LLM Output for Complex Triple Detection:")
        print("===========================================")
        print(response.choices[0].message.content)
        print("\n JSON Extraction Error:", e)

        for triple in all_extracted_triples:
            triple['chunk_id'] = str(uuid.uuid4())
            all_final_triples.append(triple)
        return all_final_triples
    complex_triples_df = pd.DataFrame(complex_triples)
    #print(f"this is complex triples cuz {complex_triples_df}")
    display(complex_triples_df)

    # STEP 2: Identify simple triples
    complex_set = {(t['subject'], t['predicate'], t['object']) for t in complex_triples}
    simple_triples = [t for t in all_extracted_triples if (t['subject'], t['predicate'], t['object']) not in complex_set]

    if not complex_triples:

      for triple in all_extracted_triples:
          triple['chunk_id'] = str(uuid.uuid4())
          all_final_triples.append(triple)
      return all_final_triples

    display(all_final_triples)


    # STEP 3: Simplify complex triples
    simplification_prompt = f"""
You are a knowledge extraction assistant. Your goal is to simplify SPO (Subject-Predicate-Object) triples into clear, atomic, and semantically accurate representations — without overcomplicating them.

Follow these guidelines carefully:

1. Retain the **core meaning** of each triple, but rephrase or reformat for clarity if needed.
2. Avoid using meta-language like "has a property", "has a source", or "can cause damage to" unless absolutely essential to preserve meaning.
3. If a **subject or object contains multiple distinct elements** (e.g., "Martin and Jake", "Spoon or Disc"), **split them into separate triples**, preserving the predicate.
4. If multiple actions or modifiers are present in a triple, **break them down** into atomic units.
5. Use **natural predicates** when possible (e.g., use "is damaged by" instead of "has a property").
6. For each simplified triple, include a `"complexity_type"` to indicate:
   - `"simple"`: A straightforward, atomic triple with one clear subject, predicate, and object.
   - `"compound"`: A triple with compound elements or multiple linked ideas.
   - `"implicit"`: A triple based on inferred meaning not explicitly stated in the original.
   - `"temporal"`: A triple that includes or implies time-based relationships.

Input:
Complex Triples:
{json.dumps(complex_triples, indent=2)}

Output:
Simplified Triples (JSON list format). Each triple must follow this structure:
{{
  "subject": "...",
  "predicate": "...",
  "object": "...",
  "complexity_type": "simple" | "compound" | "implicit" | "temporal"
}}
"""



    simplification_response = client.chat.completions.create(
        model=llm_model_name,
        messages=[
            {"role": "system", "content": "You are an assistant that simplifies logical triples for knowledge extraction."},
            {"role": "user", "content": simplification_prompt}
        ],
        temperature=llm_temperature,
        max_tokens=llm_max_tokens
    )

    try:
        simplified_triples = extract_json_list(simplification_response.choices[0].message.content)

        # Flatten in case the model nested some lists
        flattened_triples = []
        for t in simplified_triples:
            if isinstance(t, list):
                flattened_triples.extend(t)
            else:
                flattened_triples.append(t)
        simplified_triples = flattened_triples

    except Exception as e:
        print("Failed to extract simplified triples JSON:")
        print(simplification_response.choices[0].message.content)
        return []


    complex_df = pd.DataFrame(complex_triples)
    simplified_df = pd.DataFrame(simplified_triples)
    print(f"here is complex cousins {complex_triples}")
    print(f"here is simplified dudes {simplified_triples}")

    # STEP 4: Add chunk_id and return
    all_final_triples = []

    for triple in simple_triples + simplified_triples:
        if isinstance(triple, dict):
           triple['chunk_id'] = str(uuid.uuid4())
           all_final_triples.append(triple)
        else:
            print(" Unexpected triple format (not a dict):", triple)


    print(f"all_final_triples is as follows {all_final_triples}")

    return all_final_triples


In [ ]:
#---Preview of extracted triples---
def preview_spo_extraction(chunks, all_final_triples, failed_chunks):
    #--- Summary of Extraction (Reflecting state after the single chunk demo)

    print(f"\n--- Overall Extraction Summary---")
    print(f"Total chunks defined: {len(chunks)}")
    processed_chunks = len(chunks) - len(failed_chunks) #Approximation if loop didnt run fully
    print(f"Chunks processed (attempted): {processed_chunks + len(failed_chunks)}") #Chunks went through the loop
    print(f"Total valid triples extracted across all processed chunks: {len(all_final_triples)}")
    print(f"Number of chunks that failed API call or parsing: {len(failed_chunks)}")

    if failed_chunks:
        print("\nDetails of Failed Chunks:")
        for failure in failed_chunks:
            print(f"  Chunk {failure['chunk_number']}: Error: {failure['error']}")
            print(f"    Response (start): {failure.get('response', '')[:100]}...")
    print("-" * 25)

    # Display all extracted triples using Pandas
    print("\n--- All Extracted Triples (Before Normalization) ---")
    if all_final_triples:
        all_triples_df = pd.DataFrame(all_final_triples)
        display(all_triples_df)
    else:
        print("No triples were successfully extracted.")

    print("-" * 25)

    return processed_chunks, all_triples_df

In [ ]:
#---Normalization of triples---
def triplet_normalization_execution(all_final_triples):

    # Initialize lists and tracking variables
    normalized_triples = []
    seen_triples = set() # Tracking (S-P-O) tuples
    original_count = len(all_final_triples)
    empty_removed_count = 0
    duplicate_removed_count = 0

    print(f"Starting normalization and re-duplication of {original_count} triples")

    print("Processing triples for normalization (showing first 5 examples):")
    example_limit = 5
    processed_count = 0

    for i, triple in enumerate(all_final_triples):
        show_example = (i < example_limit)
        if show_example:
            print(f"\n--- Example {i+1} ---")
            print(f"Original Triple (Chunk {triple.get('chunk', '?')}): {triple}")

        subject_raw = triple.get('subject')
        predicate_raw = triple.get('predicate')
        object_raw = triple.get('object')
        chunk_num = triple.get('chunk', 'unknown')

        triple_valid = False
        normalized_sub, normalized_pred, normalized_obj = None, None, None

        if isinstance(subject_raw, str) and isinstance(predicate_raw, str) and isinstance(object_raw, str):
            # 1. Normalize
            normalized_sub = subject_raw.strip().lower()
            normalized_pred = re.sub(r'\s+', ' ', predicate_raw.strip().lower()).strip()
            normalized_obj = object_raw.strip().lower()
            if show_example:
                print(f"Normalized: SUB='{normalized_sub}', PRED='{normalized_pred}', OBJ='{normalized_obj}'")

            # 2. Filter Empty
            if normalized_sub and normalized_pred and normalized_obj:
                triple_identifier = (normalized_sub, normalized_pred, normalized_obj)

                # 3. De-duplicate
                if triple_identifier not in seen_triples:
                    normalized_triples.append({
                        'subject': normalized_sub,
                        'predicate': normalized_pred,
                        'object': normalized_obj,
                        'source_chunk': chunk_num
                    })
                    seen_triples.add(triple_identifier)
                    triple_valid = True
                    if show_example:
                        print("Status: Kept (New Unique Triple)")
                else:
                    duplicate_removed_count += 1
                    if show_example:
                        print("Status: Discarded (Duplicate)")
            else:
                empty_removed_count += 1
                if show_example:
                    print("Status: Discarded (Empty component after normalization)")
        else:
            empty_removed_count += 1 # Count non-string/missing as needing removal
            if show_example:
                print("Status: Discarded (Non-string or missing component)")
        processed_count += 1

    print(f"\n... Finished processing {processed_count} triples.")

    # --- Summary of Normalization ---
    print(f"\n--- Normalization & De-duplication Summary ---")
    print(f"Original extracted triple count: {original_count}")
    print(f"Triples removed (empty/invalid components): {empty_removed_count}")
    print(f"Duplicate triples removed: {duplicate_removed_count}")
    final_count = len(normalized_triples)
    print(f"Final unique, normalized triple count: {final_count}")
    print("-" * 25)

    # Display a sample of normalized triples using Pandas
    print("\n--- Final Normalized Triples ---")
    if normalized_triples:
        normalized_df = pd.DataFrame(normalized_triples)
        display(normalized_df)
    else:
        print("No valid triples remain after normalization.")

    print("-" * 25)

    return normalized_triples, normalized_df



In [ ]:
#---Creation of the Knowledge Graph---
def KG_creation(normalized_triples):

    #Creating an empty directed graph
    knowledge_graph = nx.DiGraph()

    print("Initialized an empty NetworkX DiGraph.")
    # Visualize the initial empty graph state
    print("--- Initial Graph Info ---")
    try:
        # Try the newer method first
        print(nx.info(knowledge_graph))
    except AttributeError:
        # Fallback for different NetworkX versions
        print(f"Type: {type(knowledge_graph).__name__}")
        print(f"Number of nodes: {knowledge_graph.number_of_nodes()}")
        print(f"Number of edges: {knowledge_graph.number_of_edges()}")
    print("-" * 25)

    print("Adding triples to the NetworkX graph...")

    added_edges_count = 0
    update_interval = 5 # How often to print graph info update

    if not normalized_triples:
        print("Warning: No normalized triples to add to the graph.")
    else:
        for i, triple in enumerate(normalized_triples):
            subject_node = triple['subject']
            object_node = triple['object']
            predicate_label = triple['predicate']

            # Nodes are added automatically when adding edges, but explicit calls are fine too
            # knowledge_graph.add_node(subject_node)
            # knowledge_graph.add_node(object_node)

            # Add the directed edge with the predicate as a 'label' attribute
            knowledge_graph.add_edge(subject_node, object_node, label=predicate_label)
            added_edges_count += 1

            # --- Visualize Graph Growth ---
            if (i + 1) % update_interval == 0 or (i + 1) == len(normalized_triples):
                print(f"\n--- Graph Info after adding Triple #{i+1} --- ({subject_node} -> {object_node})")
                try:
                    # Try the newer method first
                    print(nx.info(knowledge_graph))
                except AttributeError:
                    # Fallback for different NetworkX versions
                    print(f"Type: {type(knowledge_graph).__name__}")
                    print(f"Number of nodes: {knowledge_graph.number_of_nodes()}")
                    print(f"Number of edges: {knowledge_graph.number_of_edges()}")
                # For very large graphs, printing info too often can be slow. Adjust interval.

    print(f"\nFinished adding triples. Processed {added_edges_count} edges.")

    return knowledge_graph, added_edges_count

In [ ]:
#Summarization of the knowledge graph---
def KG_summary(knowledge_graph, added_edges_count):

    # --- Final Graph Statistics ---
    num_nodes = knowledge_graph.number_of_nodes()
    num_edges = knowledge_graph.number_of_edges()

    print(f"\n--- Final NetworkX Graph Summary ---")
    print(f"Total unique nodes (entities): {num_nodes}")
    print(f"Total unique edges (relationships): {num_edges}")

    if num_edges != added_edges_count and isinstance(knowledge_graph, nx.DiGraph):
        print(f"Note: Added {added_edges_count} edges, but graph has {num_edges}. DiGraph overwrites edges with same source/target. Use MultiDiGraph if multiple edges needed.")

    if num_nodes > 0:
        try:
          density = nx.density(knowledge_graph)
          print(f"Graph density: {density:.4f}")
          if nx.is_weakly_connected(knowledge_graph):
              print("The graph is weakly connected (all nodes reachable ignoring direction).")
          else:
              num_components = nx.number_weakly_connected_components(knowledge_graph)
              print(f"The graph has {num_components} weakly connected components.")
        except Exception as e:
            print(f"Could not calculate some graph metrics: {e}") # Handle potential errors on empty/small graphs
    else:
        print("Graph is empty, cannot calculate metrics.")
    print("-" * 25)

    # --- Sample Nodes ---
    print("\n--- Sample Nodes (First 10) ---")
    if num_nodes > 0:
        nodes_sample = list(knowledge_graph.nodes())[:10]
        display(pd.DataFrame(nodes_sample, columns=['Node Sample']))
    else:
        print("Graph has no nodes.")

    # --- Sample Edges ---
    print("\n--- Sample Edges (First 10 with Labels) ---")
    if num_edges > 0:
        edges_sample = []
        for u, v, data in list(knowledge_graph.edges(data=True))[:10]:
            edges_sample.append({'Source': u, 'Target': v, 'Label': data.get('label', 'N/A')})
        display(pd.DataFrame(edges_sample))
    else:
        print("Graph has no edges.")
    print("-" * 25)

    return {
        "num_nodes": num_nodes,
        "num_edges": num_edges,
        "nodes_sample": nodes_sample if num_nodes > 0 else [],
        "edges_sample": edges_sample if num_edges > 0 else [],
        "density": density if num_nodes > 0 else None,
        "is_weakly_connected": nx.is_weakly_connected(knowledge_graph) if num_nodes > 0 else None
    }


In [ ]:
#---Interactive Visualization---
def interactive_visualization(knowledge_graph):

    print("Preparing interactive visualization...")

    # --- Check Graph Validity for Visualization ---
    can_visualize = False
    if 'knowledge_graph' not in locals() or not isinstance(knowledge_graph, nx.Graph):
        print("Error: 'knowledge_graph' not found or is not a NetworkX graph.")
    elif knowledge_graph.number_of_nodes() == 0:
        print("NetworkX Graph is empty. Cannot visualize.")
    else:
        print(f"Graph seems valid for visualization ({knowledge_graph.number_of_nodes()} nodes, {knowledge_graph.number_of_edges()} edges).")
        can_visualize = True

    cytoscape_nodes = []
    cytoscape_edges = []

    if can_visualize:
        print("Converting nodes...")
        # Calculate degrees for node sizing
        node_degrees = dict(knowledge_graph.degree())
        max_degree = max(node_degrees.values()) if node_degrees else 1

        for node_id in knowledge_graph.nodes():
            degree = node_degrees.get(node_id, 0)
            # Simple scaling for node size (adjust logic as needed)
            node_size = 15 + (degree / max_degree) * 50 if max_degree > 0 else 15

            cytoscape_nodes.append({
                'data': {
                    'id': str(node_id), # ID must be string
                    'label': str(node_id).replace(' ', '\n'), # Display label (wrap spaces)
                    'degree': degree,
                    'size': node_size,
                    'tooltip_text': f"Entity: {str(node_id)}\nDegree: {degree}" # Tooltip on hover
                }
            })
        print(f"Converted {len(cytoscape_nodes)} nodes.")

        print("Converting edges...")
        edge_count = 0
        for u, v, data in knowledge_graph.edges(data=True):
            edge_id = f"edge_{edge_count}" # Unique edge ID
            predicate_label = data.get('label', '')
            cytoscape_edges.append({
                'data': {
                    'id': edge_id,
                    'source': str(u),
                    'target': str(v),
                    'label': predicate_label, # Label on edge
                    'tooltip_text': f"Relationship: {predicate_label}" # Tooltip on hover
                }
            })
            edge_count += 1
        print(f"Converted {len(cytoscape_edges)} edges.")

        # Combine into the final structure
        cytoscape_graph_data = {'nodes': cytoscape_nodes, 'edges': cytoscape_edges}

        # Visualize the converted structure (first few nodes/edges)
        print("\n--- Sample Cytoscape Node Data (First 2) ---")
        print(json.dumps(cytoscape_graph_data['nodes'][:2], indent=2))
        print("\n--- Sample Cytoscape Edge Data (First 2) ---")
        print(json.dumps(cytoscape_graph_data['edges'][:2], indent=2))
        print("-" * 25)
    else:
        print("Skipping data conversion as graph is not valid for visualization.")
        cytoscape_graph_data = {'nodes': [], 'edges': []}

    return cytoscape_graph_data, can_visualize

In [ ]:
#---Widget Creation---
def ipycytoscape_widget_creation(knowledge_graph, can_visualize, cytoscape_graph_data):
    if can_visualize:
        print("Creating ipycytoscape widget...")
        cyto_widget = ipycytoscape.CytoscapeWidget()
        print("Widget created.")

        print("Loading graph data into widget...")
        cyto_widget.graph.add_graph_from_json(cytoscape_graph_data, directed=True)
        print("Data loaded.")
    else:
        print("Skipping widget creation.")
        cyto_widget = None

    if cyto_widget:
        print("Defining enhanced colorful and interactive visual style...")
        # More vibrant and colorful styling with a modern color scheme
        visual_style = [
            {
                'selector': 'node',
                'style': {
                    'label': 'data(label)',
                    'width': 'data(size)',
                    'height': 'data(size)',
                    'background-color': '#3498db',  # Bright blue
                    'background-opacity': 0.9,
                    'color': '#ffffff',             # White text
                    'font-size': '12px',
                    'font-weight': 'bold',
                    'text-valign': 'center',
                    'text-halign': 'center',
                    'text-wrap': 'wrap',
                    'text-max-width': '100px',
                    'text-outline-width': 2,
                    'text-outline-color': '#2980b9',  # Matching outline
                    'text-outline-opacity': 0.7,
                    'border-width': 3,
                    'border-color': '#1abc9c',      # Turquoise border
                    'border-opacity': 0.9,
                    'shape': 'ellipse',
                    'transition-property': 'background-color, border-color, border-width, width, height',
                    'transition-duration': '0.3s',
                    'tooltip-text': 'data(tooltip_text)'
                }
            },
            {
                'selector': 'node:selected',
                'style': {
                    'background-color': '#e74c3c',  # Pomegranate red
                    'border-width': 4,
                    'border-color': '#c0392b',
                    'text-outline-color': '#e74c3c',
                    'width': 'data(size) * 1.2',    # Enlarge selected nodes
                    'height': 'data(size) * 1.2'
                }
            },
            {
                'selector': 'node:hover',
                'style': {
                    'background-color': '#9b59b6',  # Purple on hover
                    'border-width': 4,
                    'border-color': '#8e44ad',
                    'cursor': 'pointer',
                    'z-index': 999
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'label': 'data(label)',
                    'width': 2.5,
                    'curve-style': 'bezier',
                    'line-color': '#2ecc71',         # Green
                    'line-opacity': 0.8,
                    'target-arrow-color': '#27ae60',
                    'target-arrow-shape': 'triangle',
                    'arrow-scale': 1.5,
                    'font-size': '10px',
                    'font-weight': 'normal',
                    'color': '#2c3e50',
                    'text-background-opacity': 0.9,
                    'text-background-color': '#ecf0f1',
                    'text-background-shape': 'roundrectangle',
                    'text-background-padding': '3px',
                    'text-rotation': 'autorotate',
                    'edge-text-rotation': 'autorotate',
                    'transition-property': 'line-color, width, target-arrow-color',
                    'transition-duration': '0.3s',
                    'tooltip-text': 'data(tooltip_text)'
                }
            },
            {
                'selector': 'edge:selected',
                'style': {
                    'line-color': '#f39c12',         # Yellow-orange
                    'target-arrow-color': '#d35400',
                    'width': 4,
                    'text-background-color': '#f1c40f',
                    'color': '#ffffff',               # White text
                    'z-index': 998
                }
            },
            {
                'selector': 'edge:hover',
                'style': {
                    'line-color': '#e67e22',         # Orange on hover
                    'width': 3.5,
                    'cursor': 'pointer',
                    'target-arrow-color': '#d35400',
                    'z-index': 997
                }
            },
            {
                'selector': '.center-node',
                'style': {
                    'background-color': '#16a085',    # Teal
                    'background-opacity': 1,
                    'border-width': 4,
                    'border-color': '#1abc9c',        # Turquoise border
                    'border-opacity': 1
                }
            }
        ]

        print("Setting enhanced visual style on widget...")
        cyto_widget.set_style(visual_style)

        # Apply a better animated layout
        cyto_widget.set_layout(name='cose',
                              nodeRepulsion=5000,
                              nodeOverlap=40,
                              idealEdgeLength=120,
                              edgeElasticity=200,
                              nestingFactor=6,
                              gravity=90,
                              numIter=2500,
                              animate=True,
                              animationDuration=1000,
                              initialTemp=300,
                              coolingFactor=0.95)

        # Add a special class to main nodes (Marie Curie)
        if len(cyto_widget.graph.nodes) > 0:
            main_nodes = [node.data['id'] for node in cyto_widget.graph.nodes
                        if node.data.get('degree', 0) > 10]

            # Create gradient styles for center nodes
            for i, node_id in enumerate(main_nodes):
                # Use vibrant colors for center nodes
                center_style = {
                    'selector': f'node[id = "{node_id}"]',
                    'style': {
                        'background-color': '#9b59b6',   # Purple
                        'background-opacity': 0.95,
                        'border-width': 4,
                        'border-color': '#8e44ad',      # Darker purple border
                        'border-opacity': 1,
                        'text-outline-width': 3,
                        'text-outline-color': '#8e44ad',
                        'font-size': '14px'
                    }
                }
                visual_style.append(center_style)

            # Update the style with the new additions
            cyto_widget.set_style(visual_style)

        print("Enhanced colorful and interactive style applied successfully.")
    else:
        print("Skipping style definition.")

    return cyto_widget

In [ ]:
#---Layout Setting for the widget---
def setting_layout(cyto_widget):

    if cyto_widget:
        print("Setting layout algorithm ('cose')...")
        # cose (Compound Spring Embedder) is often good for exploring connections
        cyto_widget.set_layout(name='cose',
                              animate=True,
                              # Adjust parameters for better spacing/layout
                              nodeRepulsion=4000, # Increase repulsion
                              nodeOverlap=40,    # Increase overlap avoidance
                              idealEdgeLength=120, # Slightly longer ideal edges
                              edgeElasticity=150,
                              nestingFactor=5,
                              gravity=100,        # Increase gravity slightly
                              numIter=1500,      # More iterations
                              initialTemp=200,
                              coolingFactor=0.95,
                              minTemp=1.0)
        print("Layout set. The graph will arrange itself when displayed.")
    else:
        print("Skipping layout setting.")

In [ ]:
#---Widget Display Function---
def widget_display_function(cyto_widget):

    if cyto_widget:
        print("Displaying interactive graph widget below...")
        print("Interact: Zoom (scroll), Pan (drag background), Move Nodes (drag nodes), Hover for details.")
        display(cyto_widget)
    else:
        print("No widget to display.")

    # Add a clear separator
    print("\n" + "-" * 25 + "\nEnd of Visualization Step." + "\n" + "-" * 25)

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
#---RDF Graph Creation---
def rdf_graph(knowledge_graph):

    from rdflib import Graph, URIRef, Namespace, Literal
    from urllib.parse import quote

    # RDFLib Graph
    rdf_graph = Graph()
    EX = Namespace("http://kg.local/")
    rdf_graph.bind("ex", EX)

    for source, target, data in knowledge_graph.edges(data=True):
        predicate = data.get("label", "relatedTo")

        # URL-encode all parts to create valid URIs
        source_uri = URIRef(EX[quote(str(source))])
        predicate_uri = URIRef(EX[quote(str(predicate))])
        target_uri = URIRef(EX[quote(str(target))])

        rdf_graph.add((source_uri, predicate_uri, target_uri))



    rdf_graph.serialize("kg.ttl", format="turtle")

    #--- validation of turtle file ---

    rdf_graph = Graph()
    rdf_graph.parse("kg.ttl", format="turtle")
    print(f"Number of triples in graph: {len(rdf_graph)}")

    return rdf_graph

In [ ]:
#---SPARQL Query Generation---
def sparql_query(rdf):

    query = """
    SELECT ?subject ?predicate ?object
    WHERE {
      ?subject ?predicate ?object
    }
    LIMIT 16
    """

    results = rdf.query(query)

    for row in results:
        print(f"Subject: {row.subject}, Predicate: {row.predicate} Object {row.object}")

    df = pd.DataFrame(results, columns=["subject", "predicate", "object"])
    print(df)

    return sparql_query


In [ ]:
#---Installing Dependencies---

install_dependencies()


Installing pip packages...
Installing apt packages (poppler-utils)...


In [ ]:
import openai              # For interacting with OpenAI LLMs
import json                # For parsing and serializing LLM responses
import networkx as nx      # To create, manipulate, and analyze graph data structures
import ipycytoscape        # For interactive Cytoscape graph visualization in Jupyter notebooks
import ipywidgets          # To create and manage interactive widgets in Jupyter notebooks
import pandas as pd        # For data manipulation and tabular data structures (DataFrames)
import os                  # For accessing system environment variables like API keys
import math                # For basic and advanced mathematical operations
import re                  # For regular expression operations in text cleaning and pattern matching
import warnings            # To filter or suppress warning messages, such as deprecations
import together            # For interacting with Together AI's language models
import uuid                # For generating unique identifiers, such as chunk IDs
import re                  # (Duplicate) Used for pattern matching and text processing
from serpapi import GoogleSearch  # To perform web search queries using SerpAPI
import requests            # For making HTTP requests to fetch HTML or PDF files
from bs4 import BeautifulSoup     # For parsing and extracting content from HTML
from sentence_transformers import SentenceTransformer, util  # For semantic text embedding and similarity scoring
import PyPDF2              # For reading and extracting text from PDF files
from pdf2image import convert_from_path  # To convert PDF pages into images (for OCR)
import pytesseract         # For performing OCR (Optical Character Recognition) on images using Tesseract


def initialize_environment():


    #configuring settings for better display and fewer warnings
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    pd.set_option('display.max_rows', 100) #display more rows in pandas tables
    pd.set_option('display.max_colwidth', 150) #display more text width in pandas tabs

    print("Libraries installed successfully.")

In [ ]:
def KG_pipeline():
    # --- Setup environment and client ---
    initialize_environment()
    api_key, base_url, llm_model_name = retrieving_credentials()
    client_response_check()
    client = initialize_client(api_key, base_url, client="together")

    # --- LLM parameters ---
    llm_temperature, llm_max_tokens = llm_parameters()

    # --- Document search ---
    query = "satellite power system fault diagnosis PDF"
    links = search_satellite_power_docs(query, num_results=10)
    scored_links = rank_topic_links(links, base_topic="satellite power system fault diagnosis")

    # --- PDF text extraction with OCR fallback ---
    pdf_links = [link for link in links if link.lower().endswith('.pdf')]
    if pdf_links:
        pdf_url = pdf_links[0]  # pick first PDF link
        all_text = extract_text_from_pdf(pdf_url)
    else:
        all_text = ""
        print("No PDF link found.")

    # --- HTML text extraction fallback ---
    html_links = [link for link in links if not link.lower().endswith('.pdf')]
    if html_links:
        url = html_links[0]
        html_text = extract_text_from_html(url)
    else:
        html_text = ""
        print("No HTML link found.")

    # Prefer PDF text if available, otherwise HTML text
    para = all_text if all_text.strip() else html_text

    if para:
        para, char_count, word_count = unstructured_text(para)

    # --- Main document processing pipeline ---
    full_text = document_collection_pipeline_main()

    # --- Coreference resolution on full text ---
    # --- Coreference resolution on each chunk ---
    resolved_chunks = []
    for i, chunk in enumerate(full_text):
        print(f"Resolving corefs in chunk {i+1}/{len(full_text)}...")
        try:
            resolved = coref_prompt(chunk, client, llm_model_name, llm_temperature, llm_max_tokens)
            resolved_chunks.append(resolved)
        except Exception as e:
            print(f"Coref resolution failed on chunk {i+1}: {e}")
            resolved_chunks.append(chunk)  # fallback to original

    # Combine resolved text
    resolved_text = "\n".join(resolved_chunks)


    # --- Text chunking ---
    chunk_size, overlap, words, total_words = chunk_configuration(resolved_text)
    chunks, chunks_df = chunking_execution(chunk_size, overlap, words, total_words)

    # --- SPO extraction ---
    extraction_system_prompt, extraction_user_prompt_template = extraction_prompt(chunks)
    all_extracted_triples, failed_chunks = spo_extraction(
        extraction_user_prompt_template, extraction_system_prompt, chunks,
        llm_model_name, llm_temperature, llm_max_tokens, client
    )

    all_final_triples = extract_and_simplify_triples(all_extracted_triples, client, llm_model_name, llm_temperature, llm_max_tokens)
    all_triples_df = preview_spo_extraction(chunks, all_final_triples, failed_chunks)

    # --- Knowledge graph creation ---
    normalized_triples, normalized_df = triplet_normalization_execution(all_final_triples)
    knowledge_graph, added_edges_count = KG_creation(normalized_triples)
    summary = KG_summary(knowledge_graph, added_edges_count)

    num_nodes = summary["num_nodes"]
    num_edges = summary["num_edges"]

    cytoscape_graph_data, can_visualize = interactive_visualization(knowledge_graph)
    cyto_widget = ipycytoscape_widget_creation(knowledge_graph, can_visualize, cytoscape_graph_data)

    setting_layout(cyto_widget)
    widget_display_function(cyto_widget)

    # --- Create RDF graph and run SPARQL query ---
    rdf = rdf_graph(knowledge_graph)
    sparql_query(rdf)


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
#---calling KG_pipeline---
KG_pipeline()

Libraries installed successfully.
Intended LLM model: mistralai/Mixtral-8x7B-Instruct-v0.1
Retrieved API key: Set
Retrieved Base URL: Not Set (will use default TogetherAI)
 It seems like you are referring to the LLM (Master of Laws) model of education, which is typically offered online by many universities and institutions. Online education has become increasingly popular in recent years, as it allows students to access quality education from anywhere in the world, at their own pace and convenience.

Online LLM programs offer a range of benefits, such as flexibility, accessibility, and cost-effectiveness. They allow students to balance their work, family, and other commitments while pursuing their education. Additionally, online programs often offer a wide range of courses and specializations, giving students the opportunity to tailor their studies to their interests and career goals.

However, it is important to note that online learning requires a certain level of discipline, time ma

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...


--- Sample Edges (First 10 with Labels) ---


,Source,Target,Label
0,carlo cena,submitted,and colleagues
1,carlo cena,last revised,and colleagues
2,carlo cena,physics-informed real nvp for satellite power system fault detection,co-authored
3,fault detection methods,space sector,required
4,this paper,an ai-based fault detection methodology,proposes
5,this paper,electrical power system fault diagnosis,discusses
6,this paper,diagnostic capability for electrical power system faults,develops
7,our study,on the application of a pi real nvp model for fault detection in space systems,focuses
8,our physics-informed approach,existing methods of fault detection,outperforms
9,the use of a physics-informed loss,in addressing satellite eps sub-system faults,has a competitive advantage


-------------------------
Preparing interactive visualization...
Graph seems valid for visualization (645 nodes, 438 edges).
Converting nodes...
Converted 645 nodes.
Converting edges...
Converted 438 edges.

--- Sample Cytoscape Node Data (First 2) ---
[
  {
    "data": {
      "id": "carlo cena",
      "label": "carlo\ncena",
      "degree": 3,
      "size": 25.0,
      "tooltip_text": "Entity: carlo cena\nDegree: 3"
    }
  },
  {
    "data": {
      "id": "submitted",
      "label": "submitted",
      "degree": 1,
      "size": 18.333333333333332,
      "tooltip_text": "Entity: submitted\nDegree: 1"
    }
  }
]

--- Sample Cytoscape Edge Data (First 2) ---
[
  {
    "data": {
      "id": "edge_0",
      "source": "carlo cena",
      "target": "submitted",
      "label": "and colleagues",
      "tooltip_text": "Relationship: and colleagues"
    }
  },
  {
    "data": {
      "id": "edge_1",
      "source": "carlo cena",
      "target": "last revised",
      "label": "and colleagues",

CytoscapeWidget(cytoscape_layout={'name': 'cose', 'nodeRepulsion': 4000, 'nodeOverlap': 40, 'idealEdgeLength':…


-------------------------
End of Visualization Step.
-------------------------
Number of triples in graph: 438
Subject: http://kg.local/deep%20learning%20neural%20network, Predicate: http://kg.local/classify Object http://kg.local/type%20of%20fault
Subject: http://kg.local/the%20mnemonic%20with%20its%20top-1%20most%20related%20to%20%CB%86mas%20the%20new%20candidate%20root%20cause, Predicate: http://kg.local/repeats Object http://kg.local/kalman%20check%20criterion
Subject: http://kg.local/toy%20data%20set, Predicate: http://kg.local/consisted%20of Object http://kg.local/11%20flights%20of%20basic%20spacecraft%20housekeeping%20and%20physics%20data
Subject: http://kg.local/hk, Predicate: http://kg.local/is Object http://kg.local/measurement%20model%20matrix
Subject: http://kg.local/the%20algorithm%202%20walkdown%20diagnosis%20algorithm, Predicate: http://kg.local/uses Object http://kg.local/a%20causality%20matrix
Subject: http://kg.local/ablation%20testing, Predicate: http://kg.local/per